<a href="https://colab.research.google.com/github/domingues100/Water-Level-detection---IA/blob/main/Regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Mounting o drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Criar o resize do crop

In [ ]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tensorflow import keras 
from keras.applications import ResNet50,ResNet101
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import accuracy_score
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
import csv
import tensorflow_hub as hub
from tensorflow import keras
from keras.layers import Dense, Flatten
from sklearn.preprocessing import StandardScaler
import joblib 
import glob
from keras.models import load_model  
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import statistics
from sklearn.metrics import classification_report

KFOLD

In [ ]:
train_data = pd.read_csv(path) #path to instances_default.csv
Y = train_data[['label']]

for ind in train_data.index:
   train_data['label'][ind] = (train_data['label'][ind] - 3 ) #put the values between 0 and 4, instead of 3 and 7

Y = train_data[['label']]

In [ ]:
#directories
image_dir = #path# #whithout the last /
save_dir = #path# #with /
val_dir = #path# #without the last /
train_dir  =#path# #without the last /

#stratitifiedKfold - but, to train, this isnt necessary, because folds were created before and used in all tests
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True) 
num_epochs = 150

Create Image Data Generator

In [ ]:
idg = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.3,
                         fill_mode='nearest',
                         horizontal_flip = True,
                         rescale=1./255)

def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
#creates lists and variables
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
predicted = []
validation_data1 = []
scalers = []
fold_var = 9
n = Y.size

In [ ]:
for train_index, val_index in skf.split(np.zeros(n),Y):

  validation_data = pd.read_csv(val_dir+ str(fold_var)+".csv") #to get the folds that were used before
  training_data = pd.read_csv(train_dir+ str(fold_var)+".csv") #to get the folds that were used before

  scaler = StandardScaler()
  scaler.fit(training_data["label"].to_numpy().reshape([-1,1]))

  validation_data["normalizado"] = scaler.transform(validation_data["label"].to_numpy().reshape([-1,1]))
  training_data["normalizado"] = scaler.transform(training_data["label"].to_numpy().reshape([-1,1]))

  scalers.append(scaler)

  validation_data1.append(validation_data)

  training_data.to_csv(save_dir+"training_data"+str(fold_var)+".csv") #saves the folds used in the same directory, just per convinience
  validation_data.to_csv(save_dir+"validation_data"+str(fold_var)+".csv") #saves the folds used in the same directory, just per convinience

  train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
						       x_col = "id", y_col = "normalizado",
						       class_mode = "raw", shuffle = True)
 
  valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = image_dir,
							x_col = "id", y_col = "normalizado",
							class_mode = "raw", shuffle = False) 

#model-------------------------------------------------------------------------------------------------------------------------------------------
  model = keras.Sequential([
      hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5",
                     output_shape=[1280], 
                     trainable=False), 
      keras.layers.Dropout(0.4),
      keras.layers.Dense(1, activation='linear')])


  model.build([None, 224, 224, 3])  
    
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
# -----------------------------------------------------------------------------------------------------------------------------------------------

  model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae'])
	
  checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), monitor='val_accuracy', verbose=1, save_best_only=False, mode='max') #save wheights
  callbacks_list = [checkpoint]

  history = model.fit(train_data_generator, epochs=num_epochs, callbacks=callbacks_list, validation_data=valid_data_generator)

  model.load_weights(save_dir+"model_"+str(fold_var)+".h5") #load weights
	
  joblib.dump(scaler, save_dir +"scaler"+str(fold_var)) #saves the standart scaler to use for predictions

  tf.keras.backend.clear_session()
	
  fold_var += 1
  gc.collect()

# Load Model

Just need to run the next cell if it wasnt runned before. For example, if you just want to make predictions based on csvs, then run the directories cells and idg cell.




In [ ]:
model = keras.Sequential([
      hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5",
                     output_shape=[1280], 
                     trainable=False), 
      keras.layers.Dropout(0.4),
      keras.layers.Dense(1, activation='linear')])

model.build([None, 224, 224, 3])  
    
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae'])

In [ ]:
var = 1
accuracy = []
k=0

for k in range(10):

  validation_data = pd.read_csv(save_dir + f"validation_data{k+1}.csv") #get the same folds
  scaler = joblib.load(save_dir +"scaler"+str(var)) #load scalers

  valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = image_dir,
                x_col = "id", y_col = "normalizado",
                class_mode = "raw", shuffle = False) 

  model.load_weights(save_dir+"model_"+str(var)+".h5") #load weights

  pred = model.predict(valid_data_generator)
  pred = scaler.inverse_transform(pred)
  pred = np.squeeze(pred)

  for itens in validation_data:
    val_data = validation_data["label"] .values.tolist()

  val_data = list(map(int, val_data))  

  k += 1

  dados = {"predicted_desnormalizado": pred, "labels": val_data}
  df = pd.DataFrame(dados)
  df.to_csv(save_dir+f"Resultados/df{k}.csv")

# Prediction

In [ ]:
#gets the values of the predictions and calculates the errors

erro = []
erro2 = []
erro3 = []

for number in range(1,11):
  dados = pd.read_csv(save_dir+f"Resultados/df{k}.csv")

  for ind in dados.index:
   dados['labels'][ind] = (dados['labels'][ind] + 3 ) #back the values to normal
   dados['predicted_desnormalizado'][ind] = (dados['predicted_desnormalizado'][ind] + 3 ) #back the values to normal

  predicted = dados["predicted_desnormalizado"].tolist()
  label = dados["labels"].tolist()
  erro.append(mean_absolute_error(predicted, label)) 
  erro2.append(mean_squared_error(predicted, label))
  erro3.append(r2_score(predicted,label))

In [ ]:
df = pd.DataFrame(erro,columns = ["EMA"])
df['EMQ'] = erro2
df['r2'] = erro3
iteracoes = ["1", '2', '3', '4', '5', '6', '7', '8', '9', '10']
df["Iterações"] = iteracoes

In [ ]:
print(statistics.mean(erro))
print(statistics.mean(erro2))
print(statistics.mean(erro3))